In [1]:
import pandas as pd

In [2]:
dr = pd.read_csv('/content/riverside.csv')
dl = pd.read_csv('/content/Crime_Data_from_2020_to_Present_20241208.csv')

In [3]:
dl.rename(columns={'Crm Cd Desc': 'crimeType'}, inplace=True)

In [4]:
dur= dr['crimeType'].unique()
dul = dl['crimeType'].unique()

In [5]:
import re
import difflib
!pip install fuzzywuzzy
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

/usr/local/lib/python3.10/dist-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [13]:
!pip install sentence-transformers scikit-learn


In [14]:
def preprocess_crime(crime):
    crime = str(crime).lower()
    crime = re.sub(r'[^\w\s]', '', crime)
    crime = re.sub(r'\s+', ' ', crime)
    return crime.strip()

dr['Processed Crime Type'] = dr['crimeType'].apply(preprocess_crime)


In [15]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('all-MiniLM-L6-v2')
crime_embeddings = model.encode(dr['Processed Crime Type'].tolist())


/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [16]:
from sklearn.cluster import KMeans

n_clusters = 25
kmeans = KMeans(n_clusters=n_clusters, random_state=42)
dr['Cluster'] = kmeans.fit_predict(crime_embeddings)


In [18]:
dr.to_csv('clustered_crime_types.csv', index=False)
print(dr.groupby('Cluster')['Processed Crime Type'].apply(list))


Cluster
0     [other all other offenses, other all other off...
1     [other assaults not aggravated, other assaults...
2     [theft theft from motor vehicle, theft theft f...
3     [vandalism vandalism, vandalism vandalism, van...
4     [assault other assaults, assault other assault...
5     [theft shoplifting, theft shoplifting, theft s...
6     [drug narcotic violations, drug narcotic viola...
7     [burglary forcible entry, burglary forcible en...
8     [fraud embezzlement, fraud other fraud, fraud ...
9     [drug abuse violations, drug abuse violations ...
10    [all other larceny, all other larceny, all oth...
11    [destruction damage vandalism of property, des...
12    [other dui, other dui, other dui, other dui, o...
13    [weapons carrying possessing etc, weapons carr...
14    [simple assault, simple assault, simple assaul...
15    [motor veh theft trucks and buses, motor veh t...
16    [robbery strongarm, robbery firearm, assault s...
17    [other stolen property, other stol

In [19]:
cluster_names = {
    0: 'Other Crimes',
    1: 'Assaults',
    2: 'Motor Vehicle Theft',
    3: 'Vandalism',
    4: 'Fraud/Financial Crimes',
    5: 'Drug Violations',
    6: 'Vandalism',
    7: 'Physical Assaults',
    8: 'Burglary',
    9: 'Larceny',
    10: 'Stolen Property',
    11: 'Shoplifting',
    12: 'DUI/Alcohol Offenses',
    13: 'Weapons Offenses',
    14: 'Vehicle Theft',
    15: 'Sexual Assault',
    16: 'Shoplifting',
    17: 'Robbery',
    18: 'Narcotics',
    19: 'Theft',
    20: 'Public Disturbance',
    21: 'Forgery/Counterfeiting',
    22: 'Vehicle Part Theft',
    23: 'DUI/Alcohol Offenses',
    24: 'Prostitution/Vice',
}

dr['Cluster Name'] = dr['Cluster'].map(cluster_names)

In [21]:
dr.to_csv('clusterriverside.csv')

In [20]:
print(f"Crime Embeddings Shape: {crime_embeddings.shape}")
print(f"DataFrame Shape: {dl.shape}")


Crime Embeddings Shape: (148761, 384)
DataFrame Shape: (986873, 28)


In [23]:
dl['Processed Crime Type'] = dl['crimeType'].apply(preprocess_crime)
model = SentenceTransformer('all-MiniLM-L6-v2')
crime_embeddings_new = model.encode(dl['Processed Crime Type'].tolist())
n_clusters = 25
kmeans = KMeans(n_clusters=n_clusters, random_state=42)
dl['Cluster'] = kmeans.fit_predict(crime_embeddings_new)
print(dl.groupby('Cluster')['Processed Crime Type'].apply(list))

Cluster
0     [burglary from vehicle, burglary from vehicle,...
1     [theft plain petty 950 under, theft plain pett...
2     [battery simple assault, battery simple assaul...
3     [theft of identity, theft of identity, theft o...
4     [vandalism felony 400 over all church vandalis...
5     [vehicle stolen, bike stolen, vehicle stolen, ...
6     [robbery, robbery, robbery, robbery, robbery, ...
7     [intimate partner aggravated assault, intimate...
8     [assault with deadly weapon aggravated assault...
9     [theftgrand 95001 overexcptgunsfowllivestkprod...
10    [intimate partner simple assault, intimate par...
11    [violation of restraining order, violation of ...
12    [theft from motor vehicle grand 95001 and over...
13    [burglary, burglary, burglary, burglary, burgl...
14    [criminal threats no weapon displayed, crimina...
15    [shopliftinggrand theft 95001 over, shopliftin...
16    [brandish weapon, brandish weapon, discharge f...
17    [vandalism misdeameanor 399 or und

In [27]:
# Define the cluster names dictionary
cluster_names = {
    0: 'Other Crimes',
    1: 'Assaults',
    2: 'Motor Vehicle Theft',
    3: 'Vandalism',
    4: 'Fraud/Financial Crimes',
    5: 'Drug Violations',
    6: 'Vandalism',
    7: 'Physical Assaults',
    8: 'Burglary',
    9: 'Larceny',
    10: 'Stolen Property',
    11: 'Shoplifting',
    12: 'DUI/Alcohol Offenses',
    13: 'Weapons Offenses',
    14: 'Vehicle Theft',
    15: 'Sexual Assault',
    16: 'Shoplifting',
    17: 'Robbery',
    18: 'Narcotics',
    19: 'Theft',
    20: 'Public Disturbance',
    21: 'Forgery/Counterfeiting',
    22: 'Vehicle Part Theft',
    23: 'DUI/Alcohol Offenses',
    24: 'Prostitution/Vice',
}

dl['Cluster Name'] = dl['Cluster'].map(cluster_names)


In [28]:
dl.to_csv('lacluster.csv')